# Program name: ChIP-seq peak collection of gene set from RNA-seq experiment

- Version: 2025-09-17
- Author: Vera Laub
- Stage: under construction
- Input: .txt with RNA-Seq data of Pbx1/2 mutants with significantly called FC (DEseq2 pre-processing), .txt files with ChIP-seq data pf PBX1 in E10.5 HL (ChIPseeker pre-processing)
- Output: .bed files with ChIP-seq peaks of all genes defined in RNA-seq file (peak chr, peak start, peak stop, Gene ID); peaks for up/downregulated genes are saved in separate files; whole peaks and 200nt summit of peaks saved in separate files for subsequent analysis
- Description: This program serves the purpose of assembling Genes from an RNA-seq file (1. Input) and collecting the respective ChIP-seq peaks (2. Input). Code meant to be flexible :)!

In [71]:
# Set up list of genes used for collection of peaks

# Retrieve RNA-seq peaks file
file = open("/Users/veralaub/Documents/postdoc/bioinformatics/data/RNA-seq/RNA-seq_E10.5hindlimb_Pbx1-2_Hand2_mut/analysis/DEseq2_RNA-seq_Pbx1+2mutant_analysis_significant.txt", "r")
rna_seq_genes = file.readlines()
file.close()

print(rna_seq_genes[:10])

['gene_name\tbaseMean\tlog2FoldChange\tlfcSE\tstat\tpvalue\tpadj\tpbx_ctrl_rep1\tpbx_ctrl_rep2\tpbx_ctrl_rep3\tpbx_mutant_rep1\tpbx_mutant_rep2\tpbx_mutant_rep3\n', '0610010F05Rik\t1546.29395974483\t-0.210384926327583\t0.0690607410052917\t-3.04637516576115\t0.00231618635459445\t0.0206446456178762\t1684\t1632\t1602\t1414\t1567\t1390\n', '0610030E20Rik\t2351.42503521729\t-0.188728285120069\t0.0639592790254767\t-2.95075691902176\t0.0031699628552793\t0.0265451383159535\t2525\t2310\t2596\t2257\t2238\t2199\n', '0610040J01Rik\t404.661884798907\t0.904265206971576\t0.141299519122241\t6.39963400150908\t0.000000000155749838511623\t0.000000010016759816197\t250\t249\t337\t509\t526\t571\n', '1110032A03Rik\t449.935579020084\t0.335315492437819\t0.111198951156795\t3.01545553217503\t0.0025659356102332\t0.022472231128411\t359\t398\t422\t501\t542\t487\n', '1110032F04Rik\t324.046237532943\t-0.927391953297356\t0.14086411480276\t-6.58359266727304\t0.0000000000459214146948832\t0.00000000317806138437306\t419\t

In [72]:
# Since first line carries header for each column, this needs to be removed first
rna_seq_genes = rna_seq_genes[1:]

# Iterate over RNA-seq file and store information of genes in list, process data to remove whitepsace
for n in range(0, len(rna_seq_genes)):
    rna_seq_genes[n] = rna_seq_genes[n].strip()
    rna_seq_genes[n] = rna_seq_genes[n].split()
    rna_seq_genes[n][2] = float(rna_seq_genes[n][2])

print(rna_seq_genes[:10])

[['0610010F05Rik', '1546.29395974483', -0.210384926327583, '0.0690607410052917', '-3.04637516576115', '0.00231618635459445', '0.0206446456178762', '1684', '1632', '1602', '1414', '1567', '1390'], ['0610030E20Rik', '2351.42503521729', -0.188728285120069, '0.0639592790254767', '-2.95075691902176', '0.0031699628552793', '0.0265451383159535', '2525', '2310', '2596', '2257', '2238', '2199'], ['0610040J01Rik', '404.661884798907', 0.904265206971576, '0.141299519122241', '6.39963400150908', '0.000000000155749838511623', '0.000000010016759816197', '250', '249', '337', '509', '526', '571'], ['1110032A03Rik', '449.935579020084', 0.335315492437819, '0.111198951156795', '3.01545553217503', '0.0025659356102332', '0.022472231128411', '359', '398', '422', '501', '542', '487'], ['1110032F04Rik', '324.046237532943', -0.927391953297356, '0.14086411480276', '-6.58359266727304', '0.0000000000459214146948832', '0.00000000317806138437306', '419', '364', '478', '200', '246', '235'], ['1110038B12Rik', '3104.26

In [73]:
# Initiate 2 separate list of up-/ downregulated genes
genes_id_up = []
genes_id_down = []
j = 0

for line in range(0, len(rna_seq_genes)):
    if rna_seq_genes[j][2] > 0:
        genes_id_up.append(rna_seq_genes[j][0])
        j += 1
    elif rna_seq_genes[j][2] < 0:
        genes_id_down.append(rna_seq_genes[j][0])
        j += 1


print("Upregulated genes used for peak collection:", genes_id_up[:10])
print("Downregulated genes used for peak collection:", genes_id_down[:10])

Upregulated genes used for peak collection: ['0610040J01Rik', '1110032A03Rik', '1190002F15Rik', '1600014C10Rik', '1700017B05Rik', '1700020I14Rik', '1700020L24Rik', '1700021K19Rik', '1810011O10Rik', '2010003K11Rik']
Downregulated genes used for peak collection: ['0610010F05Rik', '0610030E20Rik', '1110032F04Rik', '1110038B12Rik', '1500012F01Rik', '1500015A07Rik', '1700108F19Rik', '1810032O08Rik', '2010111I01Rik', '2310057M21Rik']


In [74]:
# Set up list of ChIP-seq peaks to collect from

# Retrieve ChIP-seq peaks file
file = open("/Users/veralaub/Documents/postdoc/bioinformatics/results/online_tools/ChIPseeker/ChIPseeker_PBX1_ChIP-seq-peaks_E10.5hindlimb/ChIPseeker_PBX1_ChIP-seq-peaks_E10.5hindlimb.txt", "r")
chip_seq_peaks = file.readlines()
file.close()

print(chip_seq_peaks[:10])

['Chrom\tStart\tEnd\tgeneName\tdistanceToTSS\tannotation\tgeneChr\tgeneStart\tgeneEnd\tgeneLength\tgeneStrand\ttranscriptId\tgeneId\n', 'chr16\t18629436\t18629765\tCdc45\t576\tPromoter (<=1kb)\t16\t18628858\t18630341\t1484\t2\tENSMUST00000231819.2\tENSMUSG00000000028.16\n', 'chr7\t142135534\t142135771\tH19\t-1578\tPromoter (1-2kb)\t7\t142129268\t142133957\t4690\t2\tENSMUST00000299855.1\tENSMUSG00000000031.20\n', 'chrX\t159831221\t159831528\tScml2\t-33993\tDistal Intergenic\t20\t159865521\t159985029\t119509\t1\tENSMUST00000238603.2\tENSMUSG00000000037.18\n', 'chr11\t108156445\t108156763\tApoh\t-77417\tIntron (ENSMUST00000059595.11/ENSMUSG00000050965.15, intron 2 of 16)\t11\t108234180\t108286731\t52552\t1\tENSMUST00000133383.8\tENSMUSG00000000049.12\n', 'chr11\t108210823\t108211196\tApoh\t-22984\tIntron (ENSMUST00000059595.11/ENSMUSG00000050965.15, intron 2 of 16)\t11\t108234180\t108286731\t52552\t1\tENSMUST00000133383.8\tENSMUSG00000000049.12\n', 'chr4\t120287537\t120288024\tScmh1\t-163

In [75]:
# Since first line carries header for each column, this needs to be removed first
chip_seq_peaks = chip_seq_peaks[1:]

# Iterate over ChIP-seq file and store information of peaks in list, process data to remove whitepsace
for n in range(0, len(chip_seq_peaks)):
    chip_seq_peaks[n] = chip_seq_peaks[n].strip()
    chip_seq_peaks[n] = chip_seq_peaks[n].split()

print(chip_seq_peaks[:10])

[['chr16', '18629436', '18629765', 'Cdc45', '576', 'Promoter', '(<=1kb)', '16', '18628858', '18630341', '1484', '2', 'ENSMUST00000231819.2', 'ENSMUSG00000000028.16'], ['chr7', '142135534', '142135771', 'H19', '-1578', 'Promoter', '(1-2kb)', '7', '142129268', '142133957', '4690', '2', 'ENSMUST00000299855.1', 'ENSMUSG00000000031.20'], ['chrX', '159831221', '159831528', 'Scml2', '-33993', 'Distal', 'Intergenic', '20', '159865521', '159985029', '119509', '1', 'ENSMUST00000238603.2', 'ENSMUSG00000000037.18'], ['chr11', '108156445', '108156763', 'Apoh', '-77417', 'Intron', '(ENSMUST00000059595.11/ENSMUSG00000050965.15,', 'intron', '2', 'of', '16)', '11', '108234180', '108286731', '52552', '1', 'ENSMUST00000133383.8', 'ENSMUSG00000000049.12'], ['chr11', '108210823', '108211196', 'Apoh', '-22984', 'Intron', '(ENSMUST00000059595.11/ENSMUSG00000050965.15,', 'intron', '2', 'of', '16)', '11', '108234180', '108286731', '52552', '1', 'ENSMUST00000133383.8', 'ENSMUSG00000000049.12'], ['chr4', '120287

In [76]:
# Create list for ChIP-seq entire peaks with : Gene ID, chr, peak start, peak stop
j = 0
chip_seq_entire_peaks = []

for n in range(0, len(chip_seq_peaks)):
    chip_seq_entire_peaks.append([chip_seq_peaks[j][0], chip_seq_peaks[j][1], chip_seq_peaks[j][2], chip_seq_peaks[j][3]])
    j += 1

print(chip_seq_entire_peaks[:10])

[['chr16', '18629436', '18629765', 'Cdc45'], ['chr7', '142135534', '142135771', 'H19'], ['chrX', '159831221', '159831528', 'Scml2'], ['chr11', '108156445', '108156763', 'Apoh'], ['chr11', '108210823', '108211196', 'Apoh'], ['chr4', '120287537', '120288024', 'Scmh1'], ['chr4', '120339397', '120339734', 'Scmh1'], ['chr9', '57421051', '57421584', 'Cox5a'], ['chr11', '85799678', '85800301', 'Tbx4'], ['chr11', '59191676', '59192182', 'Wnt9a']]


In [77]:
# Function to retrieve ChIP-seq peak information of defined gene set

def construct_peak_collection(genes, chip_peaks, output_list):
    for g in genes:  # each gene ID
        for peak in chip_peaks:  # each ChIP-seq entry
            # adjust column index depending on where the gene name actually is in chip file
            if g == peak[3]:
                output_list.append([peak[0], peak[1], peak[2], peak[3]])
    return output_list

[['chr16', '18629436', '18629765', 'Cdc45'], ['chr7', '142135534', '142135771', 'H19'], ['chrX', '159831221', '159831528', 'Scml2'], ['chr11', '108156445', '108156763', 'Apoh'], ['chr11', '108210823', '108211196', 'Apoh'], ['chr4', '120287537', '120288024', 'Scmh1'], ['chr4', '120339397', '120339734', 'Scmh1'], ['chr9', '57421051', '57421584', 'Cox5a'], ['chr11', '85799678', '85800301', 'Tbx4'], ['chr11', '59191676', '59192182', 'Wnt9a']]


In [79]:
print("Example RNA-seq gene IDs:", genes_id_up[:10])
print("Example ChIP-seq first column:", [row[3] for row in chip_seq_peaks[:10]])

Example RNA-seq gene IDs: ['0610040J01Rik', '1110032A03Rik', '1190002F15Rik', '1600014C10Rik', '1700017B05Rik', '1700020I14Rik', '1700020L24Rik', '1700021K19Rik', '1810011O10Rik', '2010003K11Rik']
Example ChIP-seq first column: ['Cdc45', 'H19', 'Scml2', 'Apoh', 'Apoh', 'Scmh1', 'Scmh1', 'Cox5a', 'Tbx4', 'Wnt9a']


In [83]:
# Store information of ChIP-seq (whole) peaks of upregulated genes in two different lists
chip_seq_peaks_Pbx1ko_up = []
chip_seq_peaks_Pbx1ko_up = construct_peak_collection(genes_id_up, chip_seq_entire_peaks, chip_seq_peaks_Pbx1ko_up)
print(chip_seq_peaks_Pbx1ko_up[:10])


# Save information on entire peaks of upregulated genes in output file
chip_seq_peaks_Pbx1ko_up_output = open("/Users/veralaub/Documents/postdoc/bioinformatics/results/integration_ChIP-seq_RNA-seq_E10.5HL/ChIP-seq_PBX1_RNA-seq_Pbx1-2_mutants/PBX1chip_seq_entire_peaks_Pbx1-2ko_up.bed", "w")


def reconstruct_peak_file(chr_peaks_ko_targets):
    k = 0
    for line in range(len(chr_peaks_ko_targets)):
        chip_seq_peaks_Pbx1ko_up_output.write(chr_peaks_ko_targets[k][0] + "\t" + chr_peaks_ko_targets[k][1] + "\t" + chr_peaks_ko_targets[k][2]  + "\t" + chr_peaks_ko_targets[k][3] + "\n")
        k += 1
    return chip_seq_peaks_Pbx1ko_up_output

reconstruct_peak_file(chip_seq_peaks_Pbx1ko_up)
chip_seq_peaks_Pbx1ko_up_output.close()



# Store information of ChIP-seq entire peaks of downregulated genes in two different lists
chip_seq_peaks_Pbx1ko_down = []
chip_seq_peaks_Pbx1ko_down = construct_peak_collection(genes_id_down, chip_seq_entire_peaks, chip_seq_peaks_Pbx1ko_down)



# Save information on (whole) peaks of downregulated genes in output file
chip_seq_peaks_Pbx1ko_down_output = open("/Users/veralaub/Documents/postdoc/bioinformatics/results/integration_ChIP-seq_RNA-seq_E10.5HL/ChIP-seq_PBX1_RNA-seq_Pbx1-2_mutants/PBX1chip_seq_entire_peaks_Pbx1-2ko_down.bed", "w")

def reconstruct_peak_file(chr_peaks_ko_targets):
    k = 0
    for line in range(len(chr_peaks_ko_targets)):
        chip_seq_peaks_Pbx1ko_down_output.write(chr_peaks_ko_targets[k][0] + "\t" + chr_peaks_ko_targets[k][1] + "\t" + chr_peaks_ko_targets[k][2]  + "\t" + chr_peaks_ko_targets[k][3] + "\n")
        k += 1
    return chip_seq_peaks_Pbx1ko_down_output

reconstruct_peak_file(chip_seq_peaks_Pbx1ko_down)
chip_seq_peaks_Pbx1ko_down_output.close()

[['chr5', '63968679', '63969266', '0610040J01Rik'], ['chr9', '57158115', '57158377', '1700017B05Rik'], ['chr12', '73607980', '73608484', '2210039B01Rik'], ['chr13', '77438741', '77439220', '2210408I21Rik'], ['chr1', '9699998', '9700778', '2610203C22Rik'], ['chr17', '13844236', '13844699', '2700054A10Rik'], ['chr5', '113310457', '113311029', '2900026A02Rik'], ['chr5', '45110969', '45111233', '4930431F12Rik'], ['chr5', '45121683', '45122042', '4930431F12Rik'], ['chr8', '60983011', '60983596', 'Aadat']]


In [86]:
# General function to write only unique gene IDs from a list of peaks
def write_gene_list(peaks_list, output_path):
    # peaks_list format: [gene, chr, start, stop]
    unique_genes = sorted(set([row[3] for row in peaks_list]))
    
    with open(output_path, "w") as out:
        for gene in unique_genes:
            out.write(gene + "\n")
    print(f"File written: {output_path}")


# === Write gene list for UP genes ===
genes_up_file = "/Users/veralaub/Documents/postdoc/bioinformatics/results/integration_ChIP-seq_RNA-seq_E10.5HL/ChIP-seq_PBX1_RNA-seq_Pbx1-2_mutants/Pbx1-2ko_upregulated_genes_wChIP-peak.txt"
write_gene_list(chip_seq_peaks_Pbx1ko_up, genes_up_file)

# === Write gene list for DOWN genes ===
genes_down_file = "/Users/veralaub/Documents/postdoc/bioinformatics/results/integration_ChIP-seq_RNA-seq_E10.5HL/ChIP-seq_PBX1_RNA-seq_Pbx1-2_mutants/Pbx1-2ko_downregulated_geneswChIP-peak.txt"
write_gene_list(chip_seq_peaks_Pbx1ko_down, genes_down_file)


File written: /Users/veralaub/Documents/postdoc/bioinformatics/results/integration_ChIP-seq_RNA-seq_E10.5HL/ChIP-seq_PBX1_RNA-seq_Pbx1-2_mutants/Pbx1-2ko_upregulated_genes_wChIP-peak.txt
File written: /Users/veralaub/Documents/postdoc/bioinformatics/results/integration_ChIP-seq_RNA-seq_E10.5HL/ChIP-seq_PBX1_RNA-seq_Pbx1-2_mutants/Pbx1-2ko_downregulated_geneswChIP-peak.txt


In [ ]:
# Set up list of ChIP-seq peaks (q < 1E-20 peak call in ChIP-Atlas), downloaded form ChIP-Atlas and pre-processed with ChIPseeker via Galaxy 

# Retrieve ChIP-seq peaks file
file = open("/Users/veralaub/Documents/postdoc/bioinformatics/results/online_tools/ChIPseeker/ChIPseeker_PBX1_ChIP-seq-peaks_E10.5hindlimb/ChIPseeker_PBX1_ChIP-seq-peaks_E10.5hindlimb.txt", "r")
chip_seq_q20_peaks = file.readlines()
file.close()

print(chip_seq_200nt_peaks[:10])

In [ ]:
# Create list for ChIP-seq peaks with : Gene ID, chr, peak start, peak stop
j = 0
chip_seq_q20_peaks = []

for n in range(0, len(chip_seq_peaks)):
    chip_seq_q20_peaks.append([chip_seq_peaks[j][0], chip_seq_peaks[j][4], chip_seq_peaks[j][5], chip_seq_peaks[j][6]])
    j += 1

print(chip_seq_q20_peaks[:10])


# Store information of ChIP-seq (q20) peaks of upregulated genes in two different lists
chip_seq_peaks_q20_Pbx1ko_up = []
chip_seq_peaks_q20_Pbx1ko_up = construct_peak_collection(genes_id_up, chip_seq_q20_peaks, chip_seq_peaks_q20_Pbx1_2ko_up)
print(chip_seq_peaks_q20_Pbx1_2ko_up[:10])


# Save information on q < 1E-20 peaks of upregulated genes in output file
chip_seq_peaks_q20_Pbx1ko_up_output = open("pbx1_chip_rna_pbx1kd/chip_seq_200nt_peaks_Pbx1kd_up.bed", "w")

def reconstruct_peak_file(chr_peaks_kd_targets):
    k = 0
    for line in range(len(chr_peaks_kd_targets)):
        chip_seq_peaks_q20_Pbx1_2ko_up_output.write(chr_peaks_ko_targets[k][0] + "\t" + chr_peaks_ko_targets[k][1] + "\t" + chr_peaks_ko_targets[k][2]  + "\t" + chr_peaks_kd_targets[k][3] + "\n")
        k += 1
    return chip_seq_peaks_q20_Pbx1_2ko_up_output

reconstruct_peak_file(chip_seq_peaks_q20_Pbx1_2ko_up)
chip_seq_peaks_q20_Pbx1_2ko_up_output.close()


# Store information of ChIP-seq (200nt summit) peaks of downregulated genes in two different lists
chip_seq_peaks_q20_Pbx1_2koPbx1kd_down = []
chip_seq_peaks_q20_Pbx1_2ko_down = construct_peak_collection(genes_id_down, chip_seq_q20_peaks, chip_seq_peaks_q20_Pbx1_2ko_down)



# Save information on 200nt summit peaks of downregulated genes in output file
chip_seq_peaks_q20_Pbx1kd_down_output = open("pbx1_chip_rna_pbx1kd/chip_seq_200nt_peaks_Pbx1kd_down.bed", "w")

def reconstruct_peak_file(chr_peaks_ko_targets):
    k = 0
    for line in range(len(chr_peaks_kd_targets)):
        chip_seq_peaks_q20_Pbx1_2ko_down_output.write(chr_peaks_ko_targets[k][0] + "\t" + chr_peaks_ko_targets[k][1] + "\t" + chr_peaks_k0_targets[k][2]  + "\t" + chr_peaks_kd_targets[k][3] + "\n")
        k += 1
    return chip_seq_peaks_q20_Pbx1_2ko_down_output

reconstruct_peak_file(chip_seq_peaks_q20_Pbx1_2ko_down)
chip_seq_peaks_q20_Pbx1_2ko_down_output.close()

[['Gm10655', 'chr9', '61370468', '61370668'], ['Xpo1', 'chr11', '23255740', '23255940'], ['D930016D06Rik', 'chr5', '104571465', '104571665'], ['Slc39a3', 'chr10', '81037299', '81037499'], ['Sp2', 'chr11', '96977668', '96977868'], ['Actr2', 'chr11', '20112899', '20113099'], ['Dcaf4', 'chr12', '83520367', '83520567'], ['Acap2', 'chr16', '31201137', '31201337'], ['Nek7', 'chr1', '138618062', '138618262'], ['Fbxl19', 'chr7', '127768904', '127769104']]
[['chr1', '173942666', '173942866', 'Ifi203'], ['chr6', '17968913', '17969113', 'Wnt2'], ['chr6', '18029846', '18030046', 'Wnt2'], ['chr6', '18031674', '18031874', 'Wnt2'], ['chr19', '16593447', '16593647', 'Gna14'], ['chr19', '16484858', '16485058', 'Gna14'], ['chr19', '16540895', '16541095', 'Gna14'], ['chr19', '16598958', '16599158', 'Gna14'], ['chr19', '16442097', '16442297', 'Gna14'], ['chr19', '16529349', '16529549', 'Gna14']]
